<h1 align=center><font size = 5> Logistic Regression in TensorFlow </font></h1>

## What is the difference between Linear regression and Logistic Regression?

While Linear Regression is suited for estimating continuous values (e.g. estimating house price), it is not the best tool for predicting the class in which an observed data point belongs. In order to provide estimate for classification, we need some sort of guidance on what would be the **most probable class** for that data point. For this, we use **Logistic Regression**.

Logistic Regression is a variation of Linear Regression, useful when the observed dependent variable, __y__, is categorical. It produces a formula that predicts the probability of the class label as a function of the independent variables.

Despite the name logistic regression, it is actually a **probabilistic classification model**. Logistic regression fits a special s-shaped curve by taking the linear regression and transforming the numeric estimate into a probability with the following function:



$$
ProbabilityOfaClass = y = \frac{1}{1+e^{-z}}
$$



<img src='https://ibm.box.com/shared/static/kgv9alcghmjcv97op4d6onkyxevk23b1.png' width='400', align='center'>

#  Using logistic regression in TensorFlow 

 We are going to exemplify how to perform a logistic regression using the data from the MINST dataset. 


In [1]:
import tensorflow as tf

# We imnport MNIST dataset
from tensorflow.examples.tutorials.mnist import input_data

C:\Users\CristeamCaiolaPasqui\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



 Now we import the dataset, in this case it is a set of images of handwritten digits. 


In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

W0907 22:11:54.246982 31376 deprecation.py:323] From <ipython-input-2-a839aeb82f4b>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0907 22:11:54.247965 31376 deprecation.py:323] From C:\Users\CristeamCaiolaPasqui\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0907 22:11:54.252964 31376 deprecation.py:323] From C:\Users\CristeamCaiolaPasqui\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz


W0907 22:12:11.647545 31376 deprecation.py:323] From C:\Users\CristeamCaiolaPasqui\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0907 22:12:11.650546 31376 deprecation.py:323] From C:\Users\CristeamCaiolaPasqui\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W0907 22:12:15.154053 31376 deprecation.py:323] From C:\Users\CristeamCaiolaPasqui\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz



 Now we define the training parameters: 


In [3]:
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

Now, we define x and y. These placeholders will hold our data (both the features andn label matrices) and help pass them along to different parts of the algorithm. You can consider placeholders as empty shells into which we insert our data. We also need to give them shapes that correspond to the shape of our data. Later, we will insert data into these placeholders by feeding the placeholders the data via a feed_dict (feed dictionary):

### ¿Why use Placeholders?  
This feature of TensorFlow allows us to create an algorithm that accepts data and knows
something about the shape of the data without knowing the amount of data going in. When we insert batches of data in training, we can easily adjust how many examples we train on in
a single step without changing the entire algorithm:

In [4]:
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

### Set model weights and bias

Much like linear regression, we need a shared variable weight matrix for logistic regression.
We initialize both W and b as tensors full of zeros. Since we are going to learn W and b, their
initial value doesn't matter too much. These variables are the objects that define the
structure of our regression model, and we can save them after they've been trained so that
we can reuse them later.
We define two TensorFlow variables as our parameters. These variables will hold the
weights and biases of our logistic regression and they will be continually updated during
training.
Notice that W has a shape of [4, 3] because we want to multiply the 4-dimensional input
vectors by it to produce 3-dimensional vectors of evidence for the difference classes. b has a
shape of [3], so we can add it to the output. Moreover, unlike our placeholders (which are
essentially empty shells waiting to be fed data), TensorFlow variables need to be initialized
with values, say, with zeros:

In [5]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

###  Logistic Regression Model
We now define our operations in order to properly run the logistic regression. Logistic regression is typically thought of as a single equation:

$$
ŷ =sigmoid(WX+b)
$$

In [6]:
pred = tf.nn.sigmoid(tf.matmul(x, W) + b)

-------------------------------------

# Training

The learning algorithm is how we search for the best weight vector ($ {\bf w} $). This search is an
optimization problem looking for the hypothesis that optimizes an error/cost measure.

 **What tells us how bad our model is?** 

 The cost or loss of the model, so we want to minimize it. 

 **What cost function are we going to use?** 

 This time we are going to use cross entropy instead of the mean square error. 

 **How are we going to minimize the cost function?** 

 By gradient descent

### Cost function

In [7]:
# Minimize error using cross entropy
cost = tf.reduce_sum(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)


 Now we create a session and initialize the variables: 


In [8]:
# Create a tensorflow session
sess = tf.Session()

# Initialize our weights and biases variables.
init_op = tf.global_variables_initializer()

# Initialize all tensorflow variables
sess.run(init_op)

### Training


 And now we execute the training loop: 


In [10]:
# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Fit training using batch data
        _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                      y: batch_ys})
        # Compute average loss
        avg_cost += c / total_batch
    # Display logs per epoch step
    if (epoch+1) % display_step == 0:
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

print("Optimization Finished!")

# Test model
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# Calculate accuracy for 3000 examples
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))

Epoch: 0001 cost= 0.004707754
Epoch: 0002 cost= 0.004544954
Epoch: 0003 cost= 0.004393467
Epoch: 0004 cost= 0.004252071
Epoch: 0005 cost= 0.004119888
Epoch: 0006 cost= 0.003995963
Epoch: 0007 cost= 0.003879538
Epoch: 0008 cost= 0.003769990
Epoch: 0009 cost= 0.003666629
Epoch: 0010 cost= 0.003568991
Epoch: 0011 cost= 0.003476609
Epoch: 0012 cost= 0.003389099
Epoch: 0013 cost= 0.003306043
Epoch: 0014 cost= 0.003227068
Epoch: 0015 cost= 0.003151936
Epoch: 0016 cost= 0.003080362
Epoch: 0017 cost= 0.003012018
Epoch: 0018 cost= 0.002946831
Epoch: 0019 cost= 0.002884414
Epoch: 0020 cost= 0.002824765
Epoch: 0021 cost= 0.002767571
Epoch: 0022 cost= 0.002712772
Epoch: 0023 cost= 0.002660159
Epoch: 0024 cost= 0.002609614
Epoch: 0025 cost= 0.002561007
Optimization Finished!
Accuracy: 0.60733336
